In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler,OrdinalEncoder,LabelEncoder,OneHotEncoder


scaler_minmax= MinMaxScaler()                                           # Create a MinMaxScaler object
scaler_standered=StandardScaler()                                       # Create a StandardScaler object
oe=OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1) # Create a OrdenalEncoder object
le=LabelEncoder()                                                       # Create a LabelEncoder object
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')       # Create a OneHotEncoder object

# Read CSV files

df=pd.read_csv("Training_Set_Values.csv")    # Read the CSV file
name_featrures=df.columns                    # Get the features name
len_features=len(name_featrures)                # Get the length of features
labels=pd.read_csv("Training_Set_Labels.csv") # Read the labels CSV file
labels.head()
df['target'] = labels['status_group']        # Add the target column to the dataframe
print(df.shape)                              # Print the shape of the dataframe
df.head()
df.info()


In [ ]:
# Label Encodings for target columns
print(df['target'].head())
print(df['target'].unique())
df['target'] = le.fit_transform(df['target'])
print(df['target'].head())
dict(zip(le.classes_, le.transform(le.classes_))) # Check the mapping of labels to numbers


In [ ]:
# to see the relationship of longitude and latitude with the target column

plt.figure(figsize=(10, 6))
categories = df['target'].unique()
colors = {0: 'green', 1: 'orange', 2: 'red'}

for category in categories:
    subset = df[df['target'] == category]
    plt.scatter(subset['longitude'], subset['latitude'], 
                c=colors[category], label=category, alpha=0.5, s=10)

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Spatial Distribution by Status')
plt.legend()
plt.show()



In [ ]:
# Columns with Nan values
nan_columns = df.columns[df.isnull().any()].tolist()           # Get the columns with NaN values
print("Columns with NaN values: ", nan_columns)                # Print the columns names with NaN values
print("Number of columns with NaN values: ", len(nan_columns)) # Print the number of columns with NaN values
#df[nan_columns].head()                                         # Print the first 5 rows of the columns with NaN values

In [ ]:
# Exploration of NaN Columns
for col in nan_columns:                                             # Loop through the columns with NaN values
    print("--------------------------------------------------")     # Print a separator line
    print("Name:",col)                                              # Print the column name
    print("Number of NaN:",df[col].isnull().sum())                  # Print the number of NaN values in the column
    print("Percentage of NaN:", df[col].isnull().sum()/len(df)*100) # Print the percentage of NaN values in the column
    print(df[col].value_counts())                                   # Print the value counts of the column


In [ ]:
# Column 01 amount_tsh (Static Head)  ## 50 percent values are zero....I think Drop this column
print('Nan count in amount_tsh',df["amount_tsh"].isnull().sum()) # Number of NaN values in the column

print(df["amount_tsh"].describe())
print("median: ", df["amount_tsh"].median()) # Median is zero...means half of the points are zero (since points are positive)
plt.figure(figsize=(8,5))
sns.boxplot(df["amount_tsh"])
sns.stripplot(df["amount_tsh"],color="red",alpha=0.5)
plt.ylim(0, 100) 
plt.show()




In [ ]:
"""
# Remove the outliers from the amount_tsh column
Q1=df["amount_tsh"].quantile(0.25) # 25th percentile
Q3=df["amount_tsh"].quantile(0.75) # 75th percentile
IQR=Q3-Q1 # Interquartile range
Lower_bound_amount_tsh=Q1-1.5*IQR # Lower bound
Upper_bound_amount_tsh=Q3+1.5*IQR # Upper bound
median_amount_tsh=df["amount_tsh"].median() # Median value

df["amount_tsh_outlier_replaced_median"]=df["amount_tsh"].apply(lambda x:x if ((x>=Lower_bound_amount_tsh) &(x<=Upper_bound_amount_tsh)) else median_amount_tsh) # Replace outliers with median value


plt.figure(figsize=(8,5))
sns.boxplot(df["amount_tsh"])
sns.stripplot(df["amount_tsh_outlier_replaced_median"], color="red", alpha=0.5)                  # Adds all points
plt.title("Boxplot of amount_tsh After Outlier Removal With Median")
plt.ylim(0, 100) 
plt.show() # Show the plot
print(df["amount_tsh_outlier_replaced_median"].describe())
"""


In [ ]:

# Column 02 date_recorded (Date when the recoed entered)
print("date_recorded")
print("NaN value count:",df["date_recorded"].isnull().sum()) # Check if there are any null values in the column
df['date_recorded'] = pd.to_datetime(df['date_recorded']).dt.year# Convert to datetime format and extract year
print(df['date_recorded'].value_counts()) # Print the value counts of the column

print('Number of Unique values before outlier removal',df['date_recorded'].nunique())
df = df[~df['date_recorded'].isin([2002, 2004])] # Remove the record for years 2002 and 2004 from the dataframe.
print('Number of Unique values after outlier removal',df['date_recorded'].nunique())
                                             #31 records will be droped

"""
df['date_recorded']=oe.fit_transform(df['date_recorded'].values.reshape(-1, 1)) # Fit and transform the column using OrdinalEncoder
print('After Transformatiom',df['date_recorded'].value_counts()) # Print the value counts of the column
"""


In [ ]:
# Column 03 funder # Not Clear how to handle this column
print("NaN value count in funder:",df["funder"].isnull().sum()) # Check if there are any null values in the column
print('Unique values:',df['funder'].nunique()) # Print the number of unique values in the column
print(df['funder'].describe()) # Print the value counts of the column


# Get top 10 most common funders
top_funders = df['funder'].value_counts().head(10).index

# Filter only top funders
df_top = df[df['funder'].isin(top_funders)]

# Plot count of target classes per funder
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
sns.countplot(x='funder', hue='target', data=df_top)
plt.xticks(rotation=45)
plt.title('Water Pump Status by Top 10 Funders')
plt.xlabel('Funder')
plt.ylabel('Count')
plt.legend(title='Status', labels=['Functional', 'Needs Repair', 'Nonfunctional'])
plt.tight_layout()
plt.show()



In [ ]:
# Column 04 gps_height (GPS height) # The column needs to be considered as data is ok, minmax normalization is used
print("--------------------------------")
print("gps_height")
print("NaN value count:",df["gps_height"].isnull().sum())                # Check if there are any null values in the column
print("Percentage of NaN:", df["gps_height"].isnull().sum()/len(df)*100) # Print the percentage of NaN values in the column
print(df["gps_height"].describe())                                       # Print the description of the column
print("Median:", df["gps_height"].median())                              # Print the median of the column
plt.figure(figsize=(8,5))
sns.histplot(df["gps_height"], bins=200, kde=True)                       # Plot the histogram of the column
plt.xlabel("GPS Height")
plt.ylabel("Frequency")
plt.title("Histogram of GPS Height")
plt.show()
plt.figure(figsize=(8,5))
sns.boxplot(y=df["gps_height"])
sns.stripplot(y=df["gps_height"], color="red", alpha=0.5)                # Adds all points
plt.title("Boxplot of GPS Height Before Normalization")
plt.show() # Show the plot

"""

df["gps_height"]=scaler_minmax.fit_transform(df["gps_height"].values.reshape(-1,1)) # Fit and transform the data using MinMaxScaler
df["gps_height"] = np.clip(df["gps_height"], 0, 1) # To ensure that the values are between 0 and 1, in case the testdata has values outside the range of training data
plt.figure(figsize=(8,5))
sns.boxplot(df["gps_height"])
sns.stripplot(df["gps_height"], color="red", alpha=0.5)  # Adds all points
plt.ylabel("GPS Height (MinMax Normalized)")
plt.title("Boxplot of GPS Height After Normalization")
plt.show() # Show the plot

"""

In [ ]:
# Column 05 installer # Not Clear how to handle this column
print("NaN value count in installer:",df["installer"].isnull().sum()) # Check if there are any null values in the column
print('Unique values:',df['installer'].nunique()) # Print the number of unique values in the column
print(df['installer'].describe()) # Print the value counts of the column
#print(df['installer'].value_counts()) # Print the value counts of the column
pd.crosstab(df['installer'],df['target']).head() # Cross tabulation of installer and status_group


In [ ]:

# Get top 10 most common installer
top_installer = df['installer'].value_counts().head(10).index

# Filter only top installer
df_top = df[df['installer'].isin(top_installer)]

# Plot count of target classes per installer
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
sns.countplot(x='installer', hue='target', data=df_top)
plt.xticks(rotation=45)
plt.title('Water Pump Status by Top 10 Installer')
plt.xlabel('Installer')
plt.ylabel('Count')
plt.legend(title='Status', labels=['Functional', 'Needs Repair', 'Nonfunctional'])
plt.tight_layout()
plt.show()

In [ ]:
"""
import plotly.express as px

# Create a scatter mapbox plot with latitude and longitude
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", 
                        color="target",  # Optional: Use if you want to color by a target column
                        title="Pump Locations before oulier removal",
                        hover_data=["latitude", "longitude"])  # Optional: Display more info

fig.update_layout(mapbox_style="open-street-map")  # Use OpenStreetMap tiles

# Use an appropriate renderer for your environment (e.g., 'notebook', 'browser', etc.)
fig.show(renderer='browser')
"""


In [ ]:
#Columns 06 longitude (GPS Coordinates) # The column needs to be considered as data is ok,outlier replaced with medain and minmax normalization
print("--------------------------------")
print("longitude")
print("NaN value count:",df["longitude"].isnull().sum())                # Check if there are any null values in the column
print(df["longitude"].describe())                                                  # Print the description of the column
print("Median:", df["longitude"].median())                              # Print the median of the column

plt.figure(figsize=(8,5))
sns.histplot(df["longitude"], bins=200, kde=True)               # Plot the histogram of the column
plt.show()
plt.figure(figsize=(8,5))
sns.boxplot(df["longitude"])
sns.stripplot(df["longitude"], color="red", alpha=0.5)                  # Adds all points
plt.title("Boxplot of Longitude Before Outlier Removal")
plt.show() # Show the plot

# Remove the outliers from the longitude column
Q1=df["longitude"].quantile(0.25) # 25th percentile
Q3=df["longitude"].quantile(0.75) # 75th percentile
IQR=Q3-Q1 # Interquartile range
Lower_bound_longitude=Q1-1.5*IQR # Lower bound
Upper_bound_longitude=Q3+1.5*IQR # Upper bound
median_longitude=df["longitude"].median() # Median value

df["longitude_outlier_replaced_median"]=df["longitude"].apply(lambda x:x if ((x>=Lower_bound_longitude) &(x<=Upper_bound_longitude)) else median_longitude) # Replace outliers with median value


plt.figure(figsize=(8,5))
sns.boxplot(df["longitude"])
sns.stripplot(df["longitude_outlier_replaced_median"], color="red", alpha=0.5)                  # Adds all points
plt.title("Boxplot of Longitude After Outlier Removal With Median")
plt.show() # Show the plot







df["longitude_outlier_replaced_median_minmax_normalized"]=scaler_minmax.fit_transform(df["longitude_outlier_replaced_median"].values.reshape(-1,1)) # Fit and transform the data using MinMaxScaler
df["longitude_outlier_replaced_median_minmax_normalized"] = np.clip(df["longitude_outlier_replaced_median_minmax_normalized"], 0, 1) # To ensure that the values are between 0 and 1, in case the testdata has values outside the range of training data


plt.figure(figsize=(8,5))
sns.boxplot(df["longitude_outlier_replaced_median_minmax_normalized"])
sns.stripplot(df["longitude_outlier_replaced_median_minmax_normalized"], color="red", alpha=0.5)  # Adds all points
plt.ylabel("longitude (Outlier Replaced by Median and MinMax Normalized)")
plt.title("Boxplot of GPS longitude After Outlier Removal and Normalization")
plt.show() # Show the plot


df["longitude"] = df["longitude_outlier_replaced_median_minmax_normalized"] # Replace original longitude with processed values

df.drop("longitude_outlier_replaced_median", axis=1, inplace=True) # Drop the intermediate column
df.drop("longitude_outlier_replaced_median_minmax_normalized", axis=1, inplace=True) # Drop the intermediate column


df.columns



In [ ]:
#Columns 07 latitude (GPS Coordinates) # The column needs to be considered as data is ok
print("--------------------------------")
print("latitude")
print("NaN value count:",df["latitude"].isnull().sum())                # Check if there are any null values in the column
print(df["latitude"].describe())                                                  # Print the description of the column
print("Median:", df["latitude"].median())                              # Print the median of the column

plt.figure(figsize=(8,5))
sns.histplot(df["latitude"], bins=200, kde=True)               # Plot the histogram of the column
plt.show()
plt.figure(figsize=(8,5))
sns.boxplot(df["latitude"])
sns.stripplot(df["latitude"], color="red", alpha=0.5)                  # Adds all points
plt.title("Boxplot of latitude")
plt.show() # Show the plot




df["latitude_minmax_normalized"]=scaler_minmax.fit_transform(df["latitude"].values.reshape(-1,1)) # Fit and transform the data using MinMaxScaler
df["latitude_minmax_normalized"] = np.clip(df["latitude_minmax_normalized"], 0, 1) # To ensure that the values are between 0 and 1, in case the testdata has values outside the range of training data


plt.figure(figsize=(8,5))
sns.boxplot(df["latitude_minmax_normalized"])
sns.stripplot(df["latitude_minmax_normalized"], color="red", alpha=0.5)  # Adds all points
plt.ylabel("Latitude MinMax Normalized)")
plt.title("Boxplot of Latitude After Normalization")
plt.show() # Show the plot


df["latitude"] = df["latitude_minmax_normalized"] # Replace original longitude with processed values
df.drop("latitude_minmax_normalized", axis=1, inplace=True) # Drop the intermediate column
df.columns



In [ ]:
count_greater_than_minus_one = df[df['latitude'] > -1].shape[0]
print(count_greater_than_minus_one)


In [ ]:
"""
import plotly.express as px

# Create a scatter mapbox plot with latitude and longitude
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", 
                        color="target",  # Optional: Use if you want to color by a target column
                        title="Pump Locations",
                        hover_data=["latitude", "longitude"])  # Optional: Display more info

fig.update_layout(mapbox_style="open-street-map")  # Use OpenStreetMap tiles

# Use an appropriate renderer for your environment (e.g., 'notebook', 'browser', etc.)
fig.show(renderer='browser')
"""

In [ ]:
# Column 08 wpt_name (Waterpoint Name)  # Not Clear how to handle this column
print("NaN value count in wpt_name:",df["wpt_name"].isnull().sum()) # Check if there are any null values in the column
print('Number of unique values',df['wpt_name'].nunique()) # Print the number of unique values the column

df['wpt_name'].value_counts() # Print the value counts of the column


In [ ]:
# Column 09 num_private # Since 75% values are zero better to drop this column
print("--------------------------------")
print(df["num_private"].describe()) # Print the description of the column
df["num_private"].isnull().sum() # Check if there are any null values in the column
plt.figure(figsize=(8,5))
sns.histplot(df["num_private"], bins=200,kde=True) # Plot the histogram of the column
plt.title("Histogram of num_private")
plt.xlabel("num_private")
plt.ylabel("Frequency")
plt.show()
plt.figure(figsize=(8,5))
plt.scatter(df.index, df['num_private'], color='blue', label='Values')
plt.xlabel('Index')
plt.ylabel('Values')
plt.title("num_private vs Index")
plt.show()
plt.figure(figsize=(8,5))
sns.boxplot(df["num_private"])
#sns.stripplot(df["num_private"], color="red", alpha=0.5)                  # Adds all points
plt.title("Boxplot of num_private")
# Labels and title
plt.xlabel('Index')
plt.ylabel('Values')
plt.title('Values vs Index')
plt.show()

In [ ]:
print("NaN value count:",df["basin"].isnull().sum())                # Check if there are any null values in the column
print(df['basin'].unique())


In [ ]:


# Column 10 basin # The column needs to be considered as data is ok..I used One Hot Encoding for this
print("NaN value count:",df["basin"].isnull().sum())                # Check if there are any null values in the column
df["basin"].describe() # Print the value counts of the column
print(df.basin.value_counts()) # Print the value counts of the column
print(df["basin"].head())
encoded_basin = ohe.fit_transform(df[['basin']])
encoded_basin_df = pd.DataFrame(encoded_basin, columns=ohe.get_feature_names_out(['basin']), index=df.index)  

print(encoded_basin_df.head(5)) # Print the first 5 rows of the encoded dataframe
df = pd.concat([df, encoded_basin_df], axis=1) # Concatenate the original dataframe with the encoded dataframe
df.drop(columns=['basin'], inplace=True) # Drop the original column
df.head()



In [ ]:
# Column 11 subvillage # Since region code and district code gives same info 
                    # so ignore this because it has NaN and string to number 
                    # conversion is needed 
print("NaN count:",df["subvillage"].isnull().sum()) # Check if there are any null values in the column
print(df["subvillage"].describe()) # Print the value counts of the column
df["subvillage"].value_counts() # Print the value counts of the column
df["subvillage"].head()

"""

df['subvillage'] = df['subvillage'].fillna('Unknown')
encoded_subvillage = ohe.fit_transform(df[['subvillage']])
encoded_subvillage_df = pd.DataFrame(encoded_subvillage, columns=ohe.get_feature_names_out(['subvillage']), index=df.index)  

print(encoded_subvillage_df.head(5)) # Print the first 5 rows of the encoded dataframe
df = pd.concat([df, encoded_subvillage_df], axis=1) # Concatenate the original dataframe with the encoded dataframe
#df.drop(columns=['region'], inplace=True) # Drop the original column
#df.head()
"""

In [ ]:
#import pandas as pd
#df=pd.read_csv("Training_Set_Values.csv")    # Read the CSV file
print(df[pd.isna(df["region"])][['subvillage','region','region_code','district_code','lga','ward']])# Print the first 10 rows of the dataframe where region is Kigoma

In [ ]:
# Column 12 region # Since region code and district code gives same info 
                    # so ignore this because string to number 
                    # conversion is needed
print("NaN count:", df["region"].isnull().sum())           # Check if there are any null values in the column
print(df["region"].describe())                             # Print the description of the column
df["region"].value_counts()                                # Print the value counts of the column

print(df['region'].nunique())



encoded_region = ohe.fit_transform(df[['region']])
encoded_region_df = pd.DataFrame(encoded_region, columns=ohe.get_feature_names_out(['region']), index=df.index)  

print(encoded_region_df.head(5)) # Print the first 5 rows of the encoded dataframe
df = pd.concat([df, encoded_region_df], axis=1) # Concatenate the original dataframe with the encoded dataframe
df.drop(columns=['region'], inplace=True) # Drop the original column
df.head()


In [ ]:
#Column 13 region_code # Since it has 27 unique codes i chose to use one hot encoding.
                    
print("NaN count",df["region_code"].isnull().sum()) # Check if there are any null values in the column
print("Unique values:",df["region_code"].nunique()) # Print the distinct values of the column
print(df["region_code"].describe()) # Print the value counts of the column
df["region_code"].value_counts() # Print the first 5 rows of the column



"""
encoded_region_code = ohe.fit_transform(df[['region_code']])
encoded_region_code_df = pd.DataFrame(encoded_region_code, columns=ohe.get_feature_names_out(['region_code']), index=df.index)  

print(encoded_region_code_df.head(5)) # Print the first 5 rows of the encoded dataframe
df = pd.concat([df, encoded_region_code_df], axis=1) # Concatenate the original dataframe with the encoded dataframe
df.drop(columns=['region_code'], inplace=True) # Drop the original column
df.head()
"""

In [ ]:
# Column 14 district_code # Since has 20 unique codes i chose to use one hot encoding.
                    
print("NaN count",df["district_code"].isnull().sum()) # Check if there are any null values in the column
print("Unique values:",df["district_code"].nunique()) # Print the distinct values of the column

print(df["district_code"].describe()) # Print the value counts of the column
print("code vs freq",df["district_code"].value_counts()) # Print the value counts of the column
df["district_code"].head() # Print the first 5 rows of the column



"""
encoded_district_code = ohe.fit_transform(df[['district_code']])
encoded_district_code_df = pd.DataFrame(encoded_district_code, columns=ohe.get_feature_names_out(['district_code']), index=df.index)  

print(encoded_district_code_df.head(5)) # Print the first 5 rows of the encoded dataframe
df = pd.concat([df, encoded_district_code_df], axis=1) # Concatenate the original dataframe with the encoded dataframe
df.drop(columns=['district_code'], inplace=True) # Drop the original column
df.head()
"""



In [ ]:
# Column 15 lga # No NaN, 125 unique values
print("NaN count:",df["lga"].isnull().sum())# Print the value counts of the column
print("Unique values:",df["lga"].nunique())# Print the value counts of the column
print(df["lga"].describe()) # Print the description of the column
print(df["lga"].head()) # Print the description of the column
#freq_encoding=df["lga"].value_counts(normalize=True) # Frequency Encoding for lga column
#df["lga"]=df["lga"].map(freq_encoding) # Map the frequencies to the original column
df["lga"].head()


encoded_lga = ohe.fit_transform(df[['lga']])
encoded_lga_df = pd.DataFrame(encoded_lga, columns=ohe.get_feature_names_out(['lga']), index=df.index)  

print(encoded_lga_df.head(5)) # Print the first 5 rows of the encoded dataframe
df = pd.concat([df, encoded_lga_df], axis=1) # Concatenate the original dataframe with the encoded dataframe
df.drop(columns=['lga'], inplace=True) # Drop the original column
df.head()


In [ ]:
# Column 16 ward # No NaN, 2092 unique values, used normalized frequency encoding
print("NaN count:",df["ward"].isnull().sum())# Print the value counts of the column
print("Unique values:",df["ward"].nunique())# Print the value counts of the column
print(df["ward"].describe()) # Print the description of the column
#print(df["ward"].head()) # Print the description of the column
#freq_encoding=df["ward"].value_counts(normalize=True) # Frequency Encoding for ward column
#df["ward_freq"]=df["ward"].map(freq_encoding) # Map the frequencies to the original column
df["ward"].head()


In [ ]:
# Column 17 population
print('NaN count:',df['population'].isnull().sum()) # Check if there are any null values in the column
#print('Unique values',df['population'].value_counts()) # Print the value counts of the column
print('Median',df['population'].median()) # Print the median of the column
print(df['population'].describe())
plt.figure(figsize=(8,5))
sns.histplot(df["population"], bins=200, kde=True) # Plot the histogram of the column
plt.figure(figsize=(8,5))
sns.boxplot(df["population"])
sns.stripplot(df["population"], color="red", alpha=0.5)                  # Adds all points
plt.ylim(-100,3000)
plt.title("Boxplot of Population Before Outlier Removal")
plt.show() # Show the plot
plt.figure(figsize=(8,5))
plt.scatter(df.index, df['population'], color='blue', label='Values')
plt.show() # Show the plot

# Remove the outliers from the population column
Q1=df["population"].quantile(0.25) # 25th percentile
Q3=df["population"].quantile(0.75) # 75th percentile
IQR=Q3-Q1 # Interquartile range
Lower_bound_population=Q1-1.5*IQR # Lower bound
Upper_bound_population=Q3+1.5*IQR # Upper bound
mean_population=df["population"].mean() # Mean value

df["population_outlier_replaced_mean"]=df["population"].apply(lambda x:x if ((x>=Lower_bound_population) & (x<=Upper_bound_population)) else mean_population) # Replace outliers with median value

plt.figure(figsize=(8,5))
sns.boxplot(y=df["population"])
sns.stripplot(y=df["population_outlier_replaced_mean"], color="red", alpha=0.5)
plt.ylim(-100,3000)
plt.title("Boxplot of Population After Outlier Removal With mean")
plt.show() # Show the plot

df["population"] = df["population_outlier_replaced_mean"] # Replace original population with processed values
df.drop("population_outlier_replaced_mean", axis=1, inplace=True) # Drop the intermediate column

"""


print('Nan count in population_outlier_replaced_NaN:',df['population_outlier_replaced_NaN'].isnull().sum()) # Check if there are any null values in the column
df['population_outlier_replaced_mean'] = df.groupby('ward')['population_outlier_replaced_NaN'].transform(
    lambda x: x.fillna(x.mean())
)
print("NaN count after imputaion: ",df['population_outlier_replaced_mean'].isnull().sum()) # Check if there are any null values in the column


"""




In [ ]:
#Column 18 public_meeting # 3328 NaN values replaced with string Unknown, One Hot Encoding used
print(df['public_meeting'].value_counts())                   # Print the value counts of the column
print('Nan count',df['public_meeting'].isnull().sum())                         # Check if there are any null values in the column)
df['public_meeting']=df['public_meeting'].fillna('Unknown')  # Fill NaN values with 'Unknown'
print(df['public_meeting'].value_counts())                   # Print the value counts of the column


df['public_meeting']=df['public_meeting'].astype(str)
encoded_public_meeting = ohe.fit_transform(df[['public_meeting']])
encoded_public_meeting_df = pd.DataFrame(encoded_public_meeting, columns=ohe.get_feature_names_out(['public_meeting']), index=df.index)  

print(encoded_public_meeting_df.head(5)) # Print the first 5 rows of the encoded dataframe
df = pd.concat([df, encoded_public_meeting_df], axis=1) # Concatenate the original dataframe with the encoded dataframe
df.drop(columns=['public_meeting'], inplace=True) # Drop the original column
df.head()



In [ ]:
# Column 19 recorded_by # All values are "GeoData Consultants Ltd" ignore this column
print('Nan count:',df['recorded_by'].isnull().sum()) # Check if there are any null values in the column
df['recorded_by'].describe() # Print the value counts of the column
print('Unique values:',df['recorded_by'].nunique()) # Print the value counts of the column
print(df['recorded_by'].describe())
df.columns


In [ ]:
# Column 20 scheme_management # could not decide about this column..drop/keep
print('Nan count',df['scheme_management'].isnull().sum())
print('Unique values',df['scheme_name'].nunique())
print(df['scheme_management'].value_counts())
print('Unique values',df['scheme_management'].nunique())

#pd.crosstab(df['scheme_name'],df['scheme_management'])
pd.set_option("display.max_rows", None)
print(df[df['scheme_management'].isna()][['scheme_management','scheme_name']]) 

In [ ]:
#Column 21 scheme_name # Nan count is 28790..almost 50 percemt so ignore the column
print('Nan count:',df['scheme_name'].isnull().sum()) # Check if there are any null values in the column
df['scheme_name'].describe() # Print the value counts of the column

In [ ]:
# Column 22 permit # NaN replaced with unknown and 1 hot encdoing is used
print('Nan count:',df['permit'].isnull().sum()) # Check if there are any null values in the column
print(df['permit'].describe())
print('value count:',df['permit'].value_counts())
df['permit']=df['permit'].fillna('Unknown')
print('Nan count:',df['permit'].isnull().sum()) # Check if there are any null values in the column
print('value count:',df['permit'].value_counts())


df['permit']=df['permit'].astype(str)
encoded_permit = ohe.fit_transform(df[['permit']])
encoded_permit_df = pd.DataFrame(encoded_permit, columns=ohe.get_feature_names_out(['permit']), index=df.index)  

print(encoded_permit_df.head(5)) # Print the first 5 rows of the encoded dataframe
df = pd.concat([df, encoded_permit_df], axis=1) # Concatenate the original dataframe with the encoded dataframe
df.drop(columns=['permit'], inplace=True) # Drop the original column
df.head()



In [ ]:
#Column 23 construction_year # 1/3 values are zero.replaced with median of nonzero values
print('Nan count:',df['construction_year'].isnull().sum()) # Check if there are any null values in the column
print(df['construction_year'].value_counts().head(6)) # Print the value counts of the column
print('unique values:',df['construction_year'].unique()) # Print the value counts of the column
#pd.crosstab(df['construction_year'],df['target'])

nonzero_median = df.loc[df['construction_year'] != 0, 'construction_year'].median()
print('Nonzero median',nonzero_median)

# Step 2: Replace 0s with that median
df['construction_year'] = df['construction_year'].replace(0, nonzero_median)
print(df['construction_year'].value_counts().head(6)) # Print the value counts of the column


In [ ]:
# Columns [extraction_type, extraction_type_group, extraction_type_class]
# Based on anlysis, Extraction type is the deatiled version of other two columns 
# extraction type with 18 caregories
# extraction_type_group with 13 caregories
# extraction_type_classwith 7 caregories

# I decided to drop two columns [extraction_type_group, extraction_type_class]
# use column ['extraction_type']
# Since 'extraction_type' has 0 NaN values and 18 unique values, use one hot encoding

print('--------------------------------------------------------')
print('Nan count',df['extraction_type'].isnull().sum()) # Check if there are any null values in the column
print('Unique values',df['extraction_type'].nunique()) # Check if there are any null values in the column
print('--------------------------------------------------------')


encoded_extraction_type = ohe.fit_transform(df[['extraction_type']])
encoded_extraction_type_df = pd.DataFrame(encoded_extraction_type, columns=ohe.get_feature_names_out(['extraction_type']), index=df.index)  

print(encoded_extraction_type_df.head(5)) # Print the first 5 rows of the encoded dataframe
df = pd.concat([df, encoded_extraction_type_df], axis=1) # Concatenate the original dataframe with the encoded dataframe
df.drop(columns=['extraction_type'], inplace=True) # Drop the original column
df.head()





In [ ]:
# Column 27, 28 ['management', 'management_group'] # No NaN Values.
# 12 unique values in management
# 5 unique values in management_group
# managment is the detailed version of management_group
# I decided to drop the column management_group and use management column only
# Used one hot encoding for both columns
                                           
print('--------------------------------------------------------')
print('Nan count in management column',df['management'].isnull().sum())                         # Check if there are any null values in the column
print('value_counts in management column',df['management'].value_counts())                      # Print the value counts of the column
print('Unique values in management column',df['management'].nunique())                          # Print the number of unique values in the column
print('--------------------------------------------------------')

encoded_management = ohe.fit_transform(df[['management']])
encoded_management_df = pd.DataFrame(encoded_management, columns=ohe.get_feature_names_out(['management']), index=df.index)  

print(encoded_management_df.head(5)) # Print the first 5 rows of the encoded dataframe
df = pd.concat([df, encoded_management_df], axis=1) # Concatenate the original dataframe with the encoded dataframe
df.drop(columns=['management'], inplace=True) # Drop the original column
df.head()


In [ ]:

# Columns 29, 30 ['payment', 'payment_type']
# No Nan values and 07 Unique categories in both columns
# Semantically same values in both columns
# After mapping(staderdization) choose one column i.e payment,
# One hot encoding is used for payment column
print(df['payment'].value_counts()) #
print('--------------------------------------------------------')
print(df['payment_type'].value_counts()) # Check if there are any null values in the column

# The values in the two columns semantically are almost same just differet wording

encoded_payment = ohe.fit_transform(df[['payment']])
encoded_payment_df = pd.DataFrame(encoded_payment, columns=ohe.get_feature_names_out(['payment']), index=df.index)  

print(encoded_payment_df.head(5)) # Print the first 5 rows of the encoded dataframe
df = pd.concat([df, encoded_payment_df], axis=1) # Concatenate the original dataframe with the encoded dataframe
df.drop(columns=['payment'], inplace=True) # Drop the original column
df.head()

In [ ]:
# Columns 31, 32 ['water_quality', 'quality_group']
# Based on the value_counts water_quality is the detailed version of quality_group
# No NaN values in both columns
# 8 categories in water_quality and 6 categories in quality_group
# I chose one hot encoding for water quality and decided to drop quality_group column
#print(df['water_quality'].head())
print('--------------------------------------------------------')
print(df['water_quality'].value_counts()) # Check if there are any null values in the column
print('Nan count in water quality:',df['water_quality'].isnull().sum()) # Check if there are any null values in the column
print('--------------------------------------------------------')

encoded_water_quality = ohe.fit_transform(df[['water_quality']])
encoded_water_quality_df = pd.DataFrame(encoded_water_quality, columns=ohe.get_feature_names_out(['water_quality']), index=df.index)  

print(encoded_water_quality_df.head(5)) # Print the first 5 rows of the encoded dataframe
df = pd.concat([df, encoded_water_quality_df], axis=1) # Concatenate the original dataframe with the encoded dataframe
df.drop(columns=['water_quality'], inplace=True) # Drop the original column
df.head()

In [ ]:
# Columns 33,34 ['quantity', 'quantity_group'] are same
# No Nan Values in both columns
# 5 categories in both columns
# I chose to keep quantity column and drop quantity_group column
# 1 hot encoding is used for quantity column

print(df['quantity'].head()) 
print(df['quantity'].value_counts()) # Check if there are any null values in the column
print('Nan count in quantity',df['quantity'].isnull().sum()) # Check if there are any null values in the column

print('---------------------------------------------------------')


encoded_quantity = ohe.fit_transform(df[['quantity']])
encoded_quantity_df = pd.DataFrame(encoded_quantity, columns=ohe.get_feature_names_out(['quantity']), index=df.index)  

print(encoded_quantity_df.head(5)) # Print the first 5 rows of the encoded dataframe
df = pd.concat([df, encoded_quantity_df], axis=1) # Concatenate the original dataframe with the encoded dataframe
df.drop(columns=['quantity'], inplace=True) # Drop the original column
df.head()


In [ ]:
# Columns 35, 36 ['source', 'source_type']
# No NaN values in both columns
# 'source' is the detailed version of 'source_type'
# 10 categories in source and 7 categories in source_type
# I chose to keep source column and drop source_type column
# 1 hot encoding is used for source column
print('---------------------------------------------------------')
print(df['source'].value_counts()) # Check if there are any null values in the column
print('Nan count in source',df['source'].isnull().sum()) # Check if there are any null values in the column
print('---------------------------------------------------------')


encoded_source = ohe.fit_transform(df[['source']])
encoded_source_df = pd.DataFrame(encoded_source, columns=ohe.get_feature_names_out(['source']), index=df.index)  

print(encoded_source_df.head(5)) # Print the first 5 rows of the encoded dataframe
df = pd.concat([df, encoded_source_df], axis=1) # Concatenate the original dataframe with the encoded dataframe
df.drop(columns=['source'], inplace=True) # Drop the original column
df.head()

In [ ]:
# Column 37 ['source_class']
# No Nan values in the column
# 3 categories in the column
# 1 hot encoding is used for source_class column
print(df['source_class'].head()) # Print the first 5 rows of the column
print('---------------------------------------------------------')
#print(pd.crosstab(df['source_class'],df['source_type'])) # Print the cross tabulation of source and source_type columns
print(df['source_class'].value_counts()) # Check if there are any null values in the column
print('Nan count in source class',df['source_class'].isnull().sum()) # Check if there are any null values in the column
print('---------------------------------------------------------')


encoded_source_class = ohe.fit_transform(df[['source_class']])
encoded_source_class_df = pd.DataFrame(encoded_source_class, columns=ohe.get_feature_names_out(['source_class']), index=df.index)  

print(encoded_source_class_df.head(5)) # Print the first 5 rows of the encoded dataframe
df = pd.concat([df, encoded_source_class_df], axis=1) # Concatenate the original dataframe with the encoded dataframe
df.drop(columns=['source_class'], inplace=True) # Drop the original column
df.head()


In [ ]:
# Columns 38, 39 ['waterpoint_type', 'waterpoint_type_group'] have same info
# 'waterpoint_type' is the detailed version of 'waterpoint_type_group'
# No NaN values in both columns
# 7 categories in waterpoint_type and 6 categories in waterpoint_type_group
# I chose to keep waterpoint_type column and drop waterpoint_type_group column
# 1 hot encoding is used for waterpoint_type column
print(df['waterpoint_type'].head()) 
print('---------------------------------------------------------')
print(df['waterpoint_type'].value_counts()) # Check if there are any null values in the column
print('Nan count in waterpoint type',df['waterpoint_type'].isnull().sum()) # Check if there are any null values in the column

print('---------------------------------------------------------')


encoded_waterpoint_type = ohe.fit_transform(df[['waterpoint_type']])
encoded_waterpoint_type_df = pd.DataFrame(encoded_waterpoint_type, columns=ohe.get_feature_names_out(['waterpoint_type']), index=df.index)  

print(encoded_waterpoint_type_df.head(5)) # Print the first 5 rows of the encoded dataframe
df = pd.concat([df, encoded_waterpoint_type_df], axis=1) # Concatenate the original dataframe with the encoded dataframe
df.drop(columns=['waterpoint_type'], inplace=True) # Drop the original column
df.head()


In [ ]:
print('Shape of original dataframe:', df.shape)
"""
columns_drop=['amount_tsh','num_private','subvillage','region','recorded_by','scheme_name',
              'extraction_type_group','extraction_type_class',
              'management_group','payment_type','quality_group','quantity_group','source_type','waterpoint_type_group',
              'funder','installer','wpt_name','lga','ward','population','scheme_management']
              
'waterpoint_type_hand pump','management_vwc','date_recorded','waterpoint_type_communal standpipe multiple',
              'extraction_type_gravity','payment_pay per bucket','water_quality_soft','district_code_1'
"""

columns_drop=['id','amount_tsh','num_private','subvillage','recorded_by','scheme_name',
              'extraction_type_group','extraction_type_class',
              'management_group','payment_type','quality_group','quantity_group','source_type','waterpoint_type_group',
              'funder','installer','wpt_name','ward','scheme_management']
df = df.drop(columns=columns_drop)
print('Shape after dropping columns:', df.shape)
#df = df.drop(columns=['region_code_13', 'region_code_14', 'management_water authority', 'region_code_21', 'district_code_8', 'region_code_4', 'region_code_2', 'region_code_18', 'quantity_unknown', 'region_code_20', 'region_code_1', 'region_code_19', 'waterpoint_type_improved spring', 'management_wua', 'region_code_12', 'region_code_5', 'source_lake', 'management_parastatal', 'region_code_3', 'management_company', 'management_water board', 'permit_Unknown', 'region_code_16', 'extraction_type_afridev', 'basin_Ruvuma / Southern Coast', 'extraction_type_india mark ii', 'management_private operator', 'basin_Wami / Ruvu', 'basin_Pangani', 'extraction_type_ksb', 'region_code_17', 'district_code_7', 'source_rainwater harvesting', 'basin_Lake Nyasa', 'basin_Rufiji', 'district_code_6', 'public_meeting_Unknown', 'basin_Lake Tanganyika', 'payment_pay when scheme fails', 'basin_Lake Rukwa', 'extraction_type_mono', 'basin_Lake Victoria', 'district_code_5', 'extraction_type_swn 80', 'extraction_type_submersible', 'source_river', 'management_wug', 'payment_pay annually', 'water_quality_salty', 'basin_Internal', 'source_class_surface', 'region_code_11', 'source_class_groundwater', 'public_meeting_False', 'district_code_4', 'source_machine dbh', 'payment_pay monthly', 'public_meeting_True', 'source_shallow well', 'payment_unknown', 'district_code_2', 'quantity_seasonal', 'district_code_3', 'water_quality_unknown', 'source_spring', 'permit_False', 'extraction_type_nira/tanira', 'permit_True', 'district_code_1', 'payment_pay per bucket', 'water_quality_soft', 'extraction_type_gravity', 'date_recorded', 'waterpoint_type_communal standpipe multiple', 'management_vwc', 'waterpoint_type_hand pump', 'waterpoint_type_communal standpipe', 'quantity_insufficient', 'payment_never pay', 'extraction_type_other', 'quantity_enough', 'waterpoint_type_other', 'population', 'construction_year', 'gps_height', 'quantity_dry', 'id', 'latitude', 'longitude']
#)

print('Shape after additional dropping columns:', df.shape)



In [ ]:
"""
from sklearn.preprocessing import StandardScaler

# Drop target and non-numeric columns if any
X = df.drop(columns=['target'])  # replace 'target' with actual target column name
y = df['target']

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.decomposition import PCA

# Keep enough components to retain 95% of variance
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

print(f"Original shape: {X.shape}, After PCA: {X_pca.shape}")

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.1, random_state=42)

# Train XGBoost
model = XGBClassifier(
    n_estimators=100,
    max_depth=11,
    learning_rate=0.1,
    subsample=0.9,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='mlogloss',   # good for multi-class
    objective='multi:softmax', # directly outputs class labels
    num_class=3,               # number of target classes
    random_state=42)

model.fit(X_train, y_train)

# Evaluate
train_acc = accuracy_score(y_train, model.predict(X_train))
test_acc = accuracy_score(y_test, model.predict(X_test))

print(f"Train Accuracy: {train_acc * 100:.2f}%")
print(f"Test Accuracy: {test_acc * 100:.2f}%")

"""

In [ ]:

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
# 1. Automatically detect target column (assumed name: 'traget_encoded')
X = df.drop(columns=['target'])  # Features
y = df['target']                 # Target (already label encoded)

# 2. Train/test split (test size = 5000 rows)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.08, random_state=42, stratify=y
)

# 3. Train Random Forest Classifier
model = RandomForestClassifier(random_state=42)


model = XGBClassifier(
    n_estimators=100,
    max_depth=11,
    learning_rate=0.1,
    subsample=0.9,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='mlogloss',   # good for multi-class
    objective='multi:softmax', # directly outputs class labels
    num_class=3,               # number of target classes
    random_state=42
)


model.fit(X_train, y_train)

# 4. Predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# 5. Accuracy scores
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# 6. Print results
print(f" Training Accuracy: {train_accuracy:.4f}")
print(f" Test Accuracy:     {test_accuracy:.4f}")


In [ ]:
Training Accuracy: 0.8737
 Test Accuracy:     0.8149

In [ ]:
# Get feature importances
importances = model.feature_importances_
feature_names = X.columns

# Create a DataFrame
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Display  important features
print(" Important Features:")
print(importance_df.head())

# Optional: Plot
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'][:10][::-1], importance_df['Importance'][:10][::-1])
plt.xlabel("Feature Importance")
plt.title("Feature importance in Random Forest")
plt.tight_layout()
plt.show()




In [ ]:
importance_df.sort_values(by='Importance').head(No_of_col_to_drop)

importances = model.feature_importances_
feature_names = X.columns
len(importances),len(importance_df['Importance']),len(importance_df['Feature'])

In [ ]:
importance_df.sort_values(by='Importance', ascending=False)[['Feature', 'Importance']]
importance_df['Feature'].head(20).tolist()
